# Extract
This part will be done in the view function, customer_aging_upload()

In [14]:
import pandas as pd

In [15]:
file = "static\home\Consumer Aging django upload test.xlsx"
df = pd.read_excel(file, skiprows=2, usecols="A, C:D, H:Q", dtype={
  "Days Old":                       "Int64",
  "Batch ID":                       object,
  "Mfg Batch ID":                   object,
  "Mfg Batch Creation":             "Int64",
  "Mfg Batch Create Complete":      "Int64",
  "Ready for Fulfillment":          "Int64",
  "Tasks in Progress":              "Int64",
  "At Printer":                     "Int64",
  "Error in Print Tasks":           "Int64",
  "Print Success":                  "Int64",
  "Error in Printing":              "Int64",
  "Total Orders (blanks)":          "Int64",
  "Work Orders":                    "Int64",
})
df.head()

,Days Old,Batch ID,Mfg Batch ID,Mfg Batch Creation,Mfg Batch Create Complete,Ready for Fulfillment,Tasks in Progress,At Printer,Error in Print Tasks,Print Success,Error in Printing,Total Orders (blanks),Work Orders
0,<NA>,5066-Y9BFXN44-PKG,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,4
1,<NA>,5041-Y969DD39-PKG,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,2
2,<NA>,5064-Y9QWTWZX-PKG,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,7
3,0,5068-Y9BN3ZRX-PKG,NaN,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,7
4,0,5068-Y9BMW9Y9-PKG,NaN,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,2


# Transform

In [16]:
# Pivot status columns to a single column
status_columns = [
    "Mfg Batch Creation",
    "Mfg Batch Create Complete",
    "Ready for Fulfillment",
    "Tasks in Progress",
    "At Printer",
    "Error in Print Tasks",
    "Print Success",
    "Error in Printing",
    "Other",
]

df.insert(loc=11, column='Other', value=1)
df["status"] = df[status_columns].idxmax(axis=1, skipna=True)
df = df.drop(columns=status_columns)
df.head()

,Days Old,Batch ID,Mfg Batch ID,Total Orders (blanks),Work Orders,status
0,<NA>,5066-Y9BFXN44-PKG,NaN,1,4,Other
1,<NA>,5041-Y969DD39-PKG,NaN,1,2,Other
2,<NA>,5064-Y9QWTWZX-PKG,NaN,1,7,Other
3,0,5068-Y9BN3ZRX-PKG,NaN,1,7,Mfg Batch Creation
4,0,5068-Y9BMW9Y9-PKG,NaN,1,2,Mfg Batch Creation


In [17]:
# Rename columns
df.columns = ["age", "wob", "batch", "order_qty", "page_qty", "status"]
df.head()

,age,wob,batch,order_qty,page_qty,status
0,<NA>,5066-Y9BFXN44-PKG,NaN,1,4,Other
1,<NA>,5041-Y969DD39-PKG,NaN,1,2,Other
2,<NA>,5064-Y9QWTWZX-PKG,NaN,1,7,Other
3,0,5068-Y9BN3ZRX-PKG,NaN,1,7,Mfg Batch Creation
4,0,5068-Y9BMW9Y9-PKG,NaN,1,2,Mfg Batch Creation


In [18]:
# Reorder columns
df = df[["wob", "batch", "status", "age", "order_qty", "page_qty"]]
df.head()

,wob,batch,status,age,order_qty,page_qty
0,5066-Y9BFXN44-PKG,NaN,Other,<NA>,1,4
1,5041-Y969DD39-PKG,NaN,Other,<NA>,1,2
2,5064-Y9QWTWZX-PKG,NaN,Other,<NA>,1,7
3,5068-Y9BN3ZRX-PKG,NaN,Mfg Batch Creation,0,1,7
4,5068-Y9BMW9Y9-PKG,NaN,Mfg Batch Creation,0,1,2


In [19]:
# Fill missing values
df.age = df.age.fillna(0)
df.order_qty = df.order_qty.fillna(1)
df.batch = df.batch.fillna('')
df.head()

,wob,batch,status,age,order_qty,page_qty
0,5066-Y9BFXN44-PKG,,Other,0,1,4
1,5041-Y969DD39-PKG,,Other,0,1,2
2,5064-Y9QWTWZX-PKG,,Other,0,1,7
3,5068-Y9BN3ZRX-PKG,,Mfg Batch Creation,0,1,7
4,5068-Y9BMW9Y9-PKG,,Mfg Batch Creation,0,1,2


In [20]:
# Add uploaded_at column
df['uploaded_at'] = pd.to_datetime("now", utc=True)
df.head()

,wob,batch,status,age,order_qty,page_qty,uploaded_at
0,5066-Y9BFXN44-PKG,,Other,0,1,4,2025-03-15 14:08:28.119925+00:00
1,5041-Y969DD39-PKG,,Other,0,1,2,2025-03-15 14:08:28.119925+00:00
2,5064-Y9QWTWZX-PKG,,Other,0,1,7,2025-03-15 14:08:28.119925+00:00
3,5068-Y9BN3ZRX-PKG,,Mfg Batch Creation,0,1,7,2025-03-15 14:08:28.119925+00:00
4,5068-Y9BMW9Y9-PKG,,Mfg Batch Creation,0,1,2,2025-03-15 14:08:28.119925+00:00


# Load

In [21]:
output = "static\home\customer_aging_import.csv"
df.to_csv(output, index=False)